In [156]:
import pandas as pd

import tabula as tby



#References
#https://pypi.org/project/tabula-py/
#https://www.youtube.com/watch?v=x6AOKcPar2M

In [157]:
df = pd.DataFrame(df)

In [161]:
df[0]
# convert PDF into CSV file
tby.convert_into(r"C:\Users\sjtal\Documents\ComparisonStudyPowerBIAndPandas\20200715-covid-19-sitrep-177.pdf", r"C:\Users\sjtal\Documents\ComparisonStudyPowerBIAndPandas\WHOCountriesCovid.csv", output_format="csv", pages=[6,7,8,9,10,11,12,13,14,15])




In [182]:
df = pd.read_csv(r"C:\Users\sjtal\Documents\ComparisonStudyPowerBIAndPandas\WHOCountriesCovid.csv", encoding='latin')

In [183]:
df.tail()
#Drop the row containing Grand Total

df[df['Reporting Country/Territory/Area'] == 'Grand total']

df.drop(df.loc[df['Reporting Country/Territory/Area']=='Grand total'].index, inplace=True)
df.drop(df.loc[df['Reporting Country/Territory/Area']=='Subtotal for all regions'].index, inplace=True)

df.drop(df.loc[(df['Total confirmed']=='cases') | (df['Total confirmed']=='Totalconfirmed') | (df['Total confirmed']=='Total confirmed') ].index, inplace=True)

In [184]:
df.head()
df[df['Total confirmed']=='cases']
df['Total confirmed'].unique()

array([nan, '298 292', '33 616', '24 988', '20 216', '15 173', '13 037',
       '10 791', '8 243', '8 181', '8 134', '6 200', '5 942', '5 446',
       '5 343', '4 321', '2 497', '2 423', '2 358', '2 283', '2 153',
       '1 902', '1 780', '1 651', '1 463', '1 434', '1 416', '1 268',
       '1 099', '1 064', '1 043', '1 040', '1 038', '1 037', '884', '864',
       '731', '541', '509', '416', '399', '343', '321', '269', '256',
       '232', '100', '64', '2 724', '599', '3 344 783', '1 884 967',
       '330 123', '319 493', '304 435', '154 277', '108 155', '103 265',
       '69 570', '49 250', '47 173', '46 305', '29 742', '28 579',
       '10 303', '9 707', '8 036', '6 727', '2 980', '2 712', '2 432',
       '989', '780', '759', '300', '133', '113', '103', '74', '37', '35',
       '23', '22', '18', '17', '10 123', '6 229', '255', '206', '203',
       '190', '150', '105', '78', '72', '44', '26', '13', '12', '9', '8',
       '6', '3', '2', '262 173', '255 769', '237 803', '104 533',
      

In [186]:
df = df.rename(columns={'Reporting Country/Territory/Area':'Country', 'Total confirmed':'Total confirmed cases', 'Total confirmed.1':'Total confirmed new cases'})
# df.columns
df.set_index('Country',inplace=True)

# 238 rows × 6 columns
pd.options.display.max_rows = 999
df['Total confirmed cases'].unique()
df['Total confirmed new cases'].unique()
df['Total deaths'].unique()
df['Total new deaths'].unique()

#Found 'Total new deaths',
df[df['Total new deaths'] == 'Total new deaths'] 
df[df['Total deaths'] == 'Total new deaths'] 


,Country,Total confirmed cases,Total confirmed new cases,Total deaths,Total new deaths,Transmission,Days since last


In [187]:
df = df.dropna(axis = 0, subset=['Total confirmed cases'])

df.shape #(226, 6)




(216, 7)

In [191]:
df['Total confirmed cases']= df['Total confirmed cases'].apply(lambda x: x.replace(' ',''))
df['Total confirmed new cases'] = df['Total confirmed new cases'].apply(lambda x: x.replace(' ',''))
df['Total deaths'] = df['Total deaths'].apply(lambda x: x.replace(' ',''))
df['Total new deaths'] = df['Total new deaths'].apply(lambda x: x.replace(' ',''))

# When facing challenges in changing data type, check if you have strings in any of the columns or rows
df['Total confirmed cases'].astype(str).astype(int)
df['Total confirmed new cases'].astype(str).astype(int)
df['Total deaths'].astype(str).astype(int)
df['Total new deaths'].astype(str).astype(int)




2      174
3       10
4        0
5       10
6        0
7        3
8        5
9        0
10       7
11       0
12       0
13       0
14       2
15       2
16       0
17       1
18       0
19       1
20       0
21       0
22       0
25       0
26       1
27       2
28       0
29       0
30       0
31       0
32       1
33       0
34       0
35       0
36       0
37       0
38       1
39       0
40       0
41       0
42       0
43       0
44       0
45       0
46       0
47       0
50       0
51       0
52       0
54       0
55       0
57     349
58     733
59     184
60      45
61     485
62     148
63       7
64      67
65      67
66      59
67      23
68       7
69      25
70      15
71      11
72       4
73       1
76       2
77       0
78       8
79       0
80       0
81       0
82       0
83       0
84       0
85       0
86       0
87       0
88       0
89       0
90       0
91       0
92       0
93       0
95       2
96       2
97       0
98       0
99       0
102      0
103      0

In [193]:
df.set_index('Country',inplace=True)
df.loc['India']

Total confirmed cases                   936181
Total confirmed new cases                29429
Total deaths                             24309
Total new deaths                           582
Transmission                 Clusters of cases
Days since last                              0
Name: India, dtype: object